In [1]:
import os
import sys
import time
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from tqdm.notebook import tqdm
from pathlib import Path
import importlib.util
from gpt4all import GPT4All
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.llms import Ollama, GPT4All as LC_GPT4All

In [2]:
spec = importlib.util.spec_from_file_location("pipeline_module", "pipeline_file.py")
pipeline_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pipeline_module)
DocumentationQA = pipeline_module.DocumentationQA
AnswerGenerator = pipeline_module.AnswerGenerator

# Функция для запроса к внешней LLM для оценки
def evaluate_with_external_llm(question, context, answer, eval_metric):
    """
    Оценивает ответ с помощью внешней LLM по указанной метрике
    
    Параметры:
    - question: вопрос пользователя
    - context: контекст, на основе которого был сгенерирован ответ
    - answer: сгенерированный ответ, который нужно оценить
    - eval_metric: метрика оценки ('faithfulness', 'answer_relevancy', 'hallucination')
    
    Возвращает:
    - score: числовая оценка по шкале 1-10
    - explanation: объяснение оценки
    """
    metric_descriptions = {
        'faithfulness': "насколько ответ соответствует и основан на предоставленном контексте, верно ли передает информацию",
        'answer_relevancy': "насколько ответ релевантен заданному вопросу, отвечает ли точно на поставленный вопрос",
        'hallucination': "содержит ли ответ информацию, которой нет в контексте, или выдумывает факты (10 - нет галлюцинаций, 1 - много галлюцинаций)"
    }
    
    prompt = f"""
    Ты должен оценить качество ответа, сгенерированного языковой моделью, по шкале от 1 до 10.

    Вопрос: {question}

    Контекст (на основе которого сгенерирован ответ):
    {context}

    Ответ модели:
    {answer}

    Оцени **{eval_metric}** ответа ({metric_descriptions[eval_metric]}) по шкале от 1 до 10, где:
    - 10: превосходное качество
    - 1: очень низкое качество

    Дай оценку только в числовом виде от 1 до 10, без дополнительных комментариев и объяснений.
    """
    
    request_json = {
        "message": prompt,
        "api_key": 'chad-ea4e58bc0ac4441ca91e1188ca33120cpsekgbal'
    }
    url = "https://ask.chadgpt.ru/api/public/gpt-4o-mini"
    
    try:
        response = requests.post(url, json=request_json)
        response.raise_for_status()
        
        result = response.json()
        response_text = result.get('text', '')
        
        # Извлекаем оценку из ответа
        score_line = [line for line in response_text.split('\n') if line.startswith('Оценка:')]
        if score_line:
            try:
                score = float(score_line[0].replace('Оценка:', '').strip())
            except:
                score = None
        else:
            score = None
        
        # Извлекаем объяснение
        explanation_lines = response_text.split('Объяснение:')
        explanation = explanation_lines[1].strip() if len(explanation_lines) > 1 else response_text
        
        return {'score': score}
    
    except Exception as e:
        print(f"Ошибка при оценке: {str(e)}")
        return {'score': None, 'explanation': f"Ошибка: {str(e)}"}

In [3]:
# Класс для тестирования модели GPT4All
class GPT4AllTester:
    def __init__(self, model_path):
        self.model_path = model_path
        self.model_name = os.path.basename(model_path)
        self.model = None
    
    def initialize(self):
        if self.model is None:
            try:
                print(f"Инициализация GPT4All модели: {self.model_name}")
                self.model = GPT4All(self.model_path)
                return True
            except Exception as e:
                print(f"Ошибка при инициализации GPT4All модели: {str(e)}")
                return False
        return True
    
    def generate(self, prompt, max_tokens=500, temp=0.7, top_k=40, top_p=0.4):
        if not self.initialize():
            return f"Не удалось инициализировать модель {self.model_name}"
        
        try:
            response = self.model.generate(
                prompt,
                max_tokens=max_tokens,
                temp=temp,
                top_k=top_k,
                top_p=top_p
            )
            return response
        except Exception as e:
            return f"Ошибка генерации: {str(e)}"

# Класс для тестирования модели HuggingFace
class HuggingFaceTester:
    def __init__(self, model_id):
        self.model_id = model_id
        self.pipeline = None
        self.model_name = model_id.split('/')[-1]
    
    def initialize(self):
        if self.pipeline is None:
            try:
                print(f"Инициализация HuggingFace модели: {self.model_id}")
                tokenizer = AutoTokenizer.from_pretrained(self.model_id)
                model = AutoModelForCausalLM.from_pretrained(
                    self.model_id, 
                    device_map="auto", 
                    load_in_8bit=True
                )
                
                pipe = pipeline(
                    "text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens=500
                )
                
                self.pipeline = HuggingFacePipeline(pipeline=pipe)
                return True
            except Exception as e:
                print(f"Ошибка при инициализации HuggingFace модели: {str(e)}")
                return False
        return True
    
    def generate(self, prompt, max_tokens=500, temp=0.7, top_k=40, top_p=0.4):
        if not self.initialize():
            return f"Не удалось инициализировать модель {self.model_id}"
        
        try:
            response = self.pipeline.invoke(
                prompt,
                {"max_tokens": max_tokens, "temperature": temp, "top_k": top_k, "top_p": top_p}
            )
            return response
        except Exception as e:
            return f"Ошибка генерации: {str(e)}"

# Класс для тестирования модели Ollama
class OllamaTester:
    def __init__(self, model_name):
        self.model_name = model_name
        self.llm = None
    
    def initialize(self):
        if self.llm is None:
            try:
                print(f"Инициализация Ollama модели: {self.model_name}")
                self.llm = Ollama(model=self.model_name)
                return True
            except Exception as e:
                print(f"Ошибка при инициализации Ollama модели: {str(e)}")
                return False
        return True
    
    def generate(self, prompt, max_tokens=500, temp=0.7, top_k=40, top_p=0.4):
        if not self.initialize():
            return f"Не удалось инициализировать модель {self.model_name}"
        
        try:
            response = self.llm.invoke(
                prompt,
                {"max_tokens": max_tokens, "temperature": temp, "top_k": top_k, "top_p": top_p}
            )
            return response
        except Exception as e:
            return f"Ошибка генерации: {str(e)}"

In [4]:
# Функция для создания модифицированного класса AnswerGenerator
def create_custom_generator(llm_tester, params):
    class CustomAnswerGenerator(AnswerGenerator):
        def __init__(self, top_k=3):
            super().__init__(top_k)
            self.model = None
            self.llm_tester = llm_tester
            self.params = params
        
        def initialize_model(self):
            # Ничего не делаем здесь, так как модель инициализируется в llm_tester
            pass
        
        def generate_answer(self, question, context_list):
            context_texts = "\n\n".join([
                f"Fragment {i+1} (from {doc.get('name', 'unknown')}, similarity: {doc.get('rerank_score', doc.get('score', 0)):.3f}):\n{doc['text']}"
                for i, doc in enumerate(context_list[:self.top_k])
            ])
            
            prompt = f"""Based on the following context, please answer the question accurately and concisely.
                        Question: {question}
                        Context:
                        {context_texts}
                        Answer: """
            
            return self.llm_tester.generate(
                prompt,
                max_tokens=self.params['max_tokens'],
                temp=self.params['temperature'],
                top_k=self.params['top_k'],
                top_p=self.params['top_p']
            )
    
    return CustomAnswerGenerator

In [5]:
# Подготовка тестовых вопросов
df = pd.read_csv('qdrant_documentation_dataset.csv')

test_questions = list(df.question.unique())

In [ ]:
# Подготовка моделей для тестирования
models_to_test = [
    {
        "type": "gpt4all",
        "name": "orca-mini-3b-gguf2-q4_0.gguf",
        "description": "GPT4All Orca Mini 3B"
    },
    {
        "type": "gpt4all",
        "name": "deepseek-coder-1.3b-instruct.Q4_K_M.gguf",
        "description": "GPT4All deepseek"
    },
    {
        "type": "gpt4all",
        "name": "qwen2-1_5b-instruct-q4_0.gguf",
        "description": "GPT4All qwen2"
    }
]

# Параметры генерации для каждой модели 
# (3 разных набора параметров для каждой модели)
parameter_sets = [
    {
        "name": "conservative",
        "max_tokens": 500,
        "temperature": 0.3,
        "top_k": 40,
        "top_p": 0.9
    },
    {
        "name": "balanced",
        "max_tokens": 500,
        "temperature": 0.7,
        "top_k": 40,
        "top_p": 0.7
    },
    {
        "name": "creative",
        "max_tokens": 500,
        "temperature": 1.0,
        "top_k": 50,
        "top_p": 0.4
    }
]


In [7]:
# Основная функция запуска тестирования
def run_evaluation():
    # Инициализируем базу знаний один раз
    print("Initializing knowledge base...")
    documentation_qa = DocumentationQA()
    documentation_qa.initialize_database()
    
    results = []
    
    # Проходим по всем моделям
    for model_config in models_to_test:
        model_type = model_config["type"]
        model_name = model_config["name"]
        description = model_config["description"]
        
        print(f"\n{'='*50}")
        print(f"Testing model: {description}")
        print(f"{'='*50}")
        
        # Инициализируем модель в зависимости от типа
        if model_type == "gpt4all":
            tester = GPT4AllTester(model_name)
        elif model_type == "huggingface":
            tester = HuggingFaceTester(model_name)
        elif model_type == "ollama":
            tester = OllamaTester(model_name)
        else:
            print(f"Неизвестный тип модели: {model_type}")
            continue
            
        # Проверяем, можем ли инициализировать модель
        if not tester.initialize():
            print(f"Пропускаем модель {description} из-за ошибки инициализации")
            continue
        
        # Проходим по всем наборам параметров
        for params in parameter_sets:
            param_name = params["name"]
            print(f"\nTesting with {param_name} parameters:")
            
            # Создаем кастомный генератор ответов
            custom_generator = create_custom_generator(tester, params)
            
            # Заменяем генератор ответов в нашем пайплайне
            documentation_qa.answer_generator = custom_generator(top_k=3)
                        # Проходим по всем тестовым вопросам
            for question in tqdm(test_questions, desc=f"{description} - {param_name}"):
                # Поиск похожих параграфов
                similar_paragraphs = documentation_qa.search_similar_paragraphs(question, top_k=3)
                
                # Формируем контекст (такой же, как будет использоваться для генерации)
                context_texts = "\n\n".join([
                    f"Fragment {i+1} (from {doc.get('name', 'unknown')}, similarity: {doc.get('rerank_score', doc.get('score', 0)):.3f}):\n{doc['text']}"
                    for i, doc in enumerate(similar_paragraphs)
                ])
                
                # Генерируем ответ
                try:
                    start_time = time.time()
                    answer = documentation_qa.answer_generator.generate_answer(question, similar_paragraphs)
                    generation_time = time.time() - start_time
                except Exception as e:
                    print(f"Ошибка при генерации ответа: {str(e)}")
                    continue
                
                # Оцениваем по каждой из метрик
                metrics = {}
                for metric in ['faithfulness', 'answer_relevancy', 'hallucination']:
                    eval_result = evaluate_with_external_llm(question, context_texts, answer, metric)
                    metrics[metric] = eval_result
                
                # Сохраняем результаты
                result = {
                    "model_type": model_type,
                    "model_name": model_name,
                    "description": description,
                    "parameters": param_name,
                    "question": question,
                    "context": context_texts,
                    "answer": answer,
                    "generation_time": generation_time,
                    "faithfulness_score": metrics['faithfulness']['score'],
                    "relevancy_score": metrics['answer_relevancy']['score'],
                    "hallucination_score": metrics['hallucination']['score'],
                }
                
                results.append(result)
                
                # Сохраняем промежуточные результаты
                df = pd.DataFrame(results)
                df.to_csv('rag_model_evaluation_results.csv', index=False)
                
                # Делаем паузу, чтобы не перегружать API
                time.sleep(2)
    
    return pd.DataFrame(results)

In [8]:
# Анализ результатов
def analyze_results(results_df):
    # Создаем сводную таблицу со средними оценками по каждой модели и набору параметров
    summary = results_df.groupby(['description', 'parameters']).agg({
        'faithfulness_score': 'mean',
        'relevancy_score': 'mean',
        'hallucination_score': 'mean',
        'generation_time': 'mean'
    }).reset_index()
    
    # Добавляем общий средний балл
    summary['average_score'] = summary[['faithfulness_score', 'relevancy_score', 'hallucination_score']].mean(axis=1)
    
    # Сортируем по среднему баллу
    summary = summary.sort_values('average_score', ascending=False)
    
    print("Сводные результаты (сортировка по общему среднему баллу):")
    print(summary)
    
    # Визуализация результатов
    plt.figure(figsize=(14, 8))
    
    # Создаем сводную таблицу для графика
    plot_data = results_df.pivot_table(
        index=['description', 'parameters'],
        values=['faithfulness_score', 'relevancy_score', 'hallucination_score'],
        aggfunc='mean'
    ).reset_index()
    
    plot_data = plot_data.melt(
        id_vars=['description', 'parameters'],
        value_vars=['faithfulness_score', 'relevancy_score', 'hallucination_score'],
        var_name='metric',
        value_name='score'
    )
    
    # Строим график
    sns.barplot(x='description', y='score', hue='metric', data=plot_data, palette='viridis')
    plt.xticks(rotation=45, ha='right')
    plt.title('Средние оценки моделей по метрикам')
    plt.tight_layout()
    plt.savefig('model_scores_by_metric.png')
    
    # График влияния параметров на каждую модель
    plt.figure(figsize=(14, 8))
    sns.catplot(
        x='parameters', 
        y='average_score', 
        hue='description', 
        kind='bar', 
        data=summary,
        height=6, 
        aspect=1.5
    )
    plt.title('Влияние параметров на производительность моделей')
    plt.tight_layout()
    plt.savefig('parameters_impact.png')
    
    # Время генерации
    plt.figure(figsize=(12, 6))
    sns.barplot(x='description', y='generation_time', data=summary, palette='rocket')
    plt.xticks(rotation=45, ha='right')
    plt.title('Среднее время генерации ответа (секунды)')
    plt.tight_layout()
    plt.savefig('generation_time.png')
    
    return summary

In [9]:
# Функция для подробного анализа конкретного вопроса
def analyze_question(results_df, question):
    question_results = results_df[results_df['question'] == question].copy()
    
    # Добавляем средний балл
    question_results['average_score'] = question_results[['faithfulness_score', 'relevancy_score', 'hallucination_score']].mean(axis=1)
    
    # Сортируем по среднему баллу
    question_results = question_results.sort_values('average_score', ascending=False)
    
    print(f"Анализ ответов на вопрос: '{question}'")
    print("\nТоп-3 лучших ответа:")
    for i, row in question_results.head(3).iterrows():
        print(f"\n{row['description']} с {row['parameters']} параметрами (средняя оценка: {row['average_score']:.2f}):")
        print(f"Ответ: {row['answer'][:300]}...")
        print(f"Оценки: faithfulness={row['faithfulness_score']}, relevancy={row['relevancy_score']}, hallucination={row['hallucination_score']}")
    
    print("\nХудший ответ:")
    worst = question_results.iloc[-1]
    print(f"{worst['description']} с {worst['parameters']} параметрами (средняя оценка: {worst['average_score']:.2f}):")
    print(f"Ответ: {worst['answer'][:300]}...")
    print(f"Оценки: faithfulness={worst['faithfulness_score']}, relevancy={worst['relevancy_score']}, hallucination={worst['hallucination_score']}")
    
    return question_results

In [ ]:
# Запуск всего процесса оценки и анализа
if __name__ == "__main__":
    # Проверяем, есть ли уже сохраненные результаты
    if os.path.exists('rag_model_evaluation_results.csv'):
        print("Найдены сохраненные результаты. Загружаем...")
        results_df = pd.read_csv('rag_model_evaluation_results.csv')
        print(f"Загружено {len(results_df)} результатов тестирования.")
    else:
        print("Запускаем процесс оценки моделей (это может занять много времени)...")
        results_df = run_evaluation()
    
    # Анализируем результаты
    summary = analyze_results(results_df)
    
    # Анализируем конкретный вопрос для примера
    sample_question = "How does indexing work in Qdrant?"
    question_analysis = analyze_question(results_df, sample_question)

Запускаем процесс оценки моделей (это может занять много времени)...
Initializing knowledge base...

Testing model: GPT4All Orca Mini 3B
Инициализация GPT4All модели: orca-mini-3b-gguf2-q4_0.gguf

Testing with conservative parameters:


GPT4All Orca Mini 3B - conservative:   0%|          | 0/327 [00:00<?, ?it/s]

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x000001DBFFEDAA20>
Traceback (most recent call last):
  File "C:\Users\sekho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gpt4all\_pyllmodel.py", line 614, in _prompt_callback
    @staticmethod

KeyboardInterrupt: 



Testing with balanced parameters:


GPT4All Orca Mini 3B - balanced:   0%|          | 0/327 [00:00<?, ?it/s]

Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini
Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini
Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini
Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini
Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini
Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini
Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini
Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini
Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.

GPT4All Orca Mini 3B - creative:   0%|          | 0/327 [00:00<?, ?it/s]

Ошибка при оценке: 500 Server Error: Internal Server Error for url: https://ask.chadgpt.ru/api/public/gpt-4o-mini


Exception ignored on calling ctypes callback function: <function LLModel._callback_decoder.<locals>._raw_callback at 0x000001DC14E72160>
Traceback (most recent call last):
  File "C:\Users\sekho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gpt4all\_pyllmodel.py", line 573, in _raw_callback
    def _raw_callback(token_id: int, response: bytes) -> bool:

KeyboardInterrupt: 
